## 参考資料
- [人工衛星の軌道をPythonでアニメーションにしてみよう](https://qiita.com/ciscorn/items/80b3a3f526316f78b24a)
- [skyfiedl Earth Satellites](https://rhodesmill.org/skyfield/earth-satellites.html)

In [ ]:
import skyfield.api
from skyfield.framelib import itrs
from skyfield.api import wgs84
from datetime import datetime, timezone, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

- GNSS軌道データを取ってくる

In [ ]:
sats = skyfield.api.load.tle_file("https://celestrak.org/NORAD/elements/gnss.txt", reload=True)
len(sats)

- timescale()オブジェクトを使う準備

In [ ]:
ts = skyfield.api.load.timescale()

In [ ]:
# タイムゾーンを指定
my_timezone = timezone(timedelta(hours=9))  # 例: UTC+9 (日本時間)

# 現在の時刻を取得してエポックタイムに変換
current_time = datetime.now()
epoch_time = current_time.timestamp()

# 切り上げたい分数（5の倍数）を計算
rounded_epoch_time = ((epoch_time + 300) // 300) * 300  # 300秒 = 5分

# エポックタイムをtimezone awareなdatetimeに変換
rounded_datetime = datetime.fromtimestamp(rounded_epoch_time, tz=my_timezone)
rounded_datetime


In [ ]:
duration_hours = 72 * 2 # interval_min間隔で生成する時間期間
interval_min = 5        # duration_hours期間中の計算生成間隔[分]
generate_count = int(duration_hours * 60 / 5)
generate_count          # 生成回数

- 現在時刻を5分単位で切り上げて、指定時間後までの5分間隔のdatetimeのリストを生成する。

In [ ]:
date_times = [rounded_datetime + timedelta(minutes=5 * i) for i in range(generate_count)]
date_times[0],date_times[-1]

- skyfieldのtimescale型に変換

In [ ]:
tss = ts.from_datetimes(date_times)
tss

- 地球の重心を原点としつつ地球の自転と一緒に回転しない座標系 (GCRS; Geocentric Celestial Reference System) の位置を計算/表示


In [ ]:
# 3Dプロットの初期化
fig = plt.figure(figsize=[16,9])
ax = fig.add_subplot(111, projection='3d')

for sat in sats:
    if sat.name.startswith("QZ"):
#    if sat.name.startswith("NAVSTAR"):
        df = pd.DataFrame([sat.at(target_ts).position.m for target_ts in tss], columns=['x','y','z'])
        ax.scatter(df['x'], df['y'], df['z'], s=10, label=sat.name)
#        ax.scatter(df['x'], df['y'], df['z'], s=10, label="_".join(sat.name.split()[:2]))


# 軸ラベルの設定
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel
plt.legend()



- 地球に固定された座標系ITRS/ITRF（国際地球基準座標系）で位置を計算/表示

In [ ]:
# 3Dプロットの初期化
fig = plt.figure(figsize=[16,9])
ax = fig.add_subplot(111, projection='3d')

for sat in sats:
    if sat.name.startswith("QZ"):
#    if sat.name.startswith("NAVSTAR"):
        df = pd.DataFrame([sat.at(target_ts).frame_xyz(itrs).m for target_ts in tss], columns=['x','y','z'])
        ax.scatter(df['x'], df['y'], df['z'], s=10, label=sat.name)
#        ax.scatter(df['x'], df['y'], df['z'], s=10, label="_".join(sat.name.split()[:2]))


# 軸ラベルの設定
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel
plt.legend()


- 衛星位置を緯度経度で求めて表示

In [ ]:
for sat in sats:
    if sat.name.startswith("QZ"):
        df = pd.DataFrame([{'lat':wgs84.latlon_of(sat.at(target_ts))[0].degrees,'lon':wgs84.latlon_of(sat.at(target_ts))[1].degrees} for target_ts in tss])
        plt.plot(df['lon'], df['lat'], label=sat.name, alpha=0.5)
plt.legend()


- 指定位置から見た天空図を生成

In [ ]:
home = wgs84.latlon(+35.400334,+139.543152)

fig = plt.figure(figsize=[16,9])
ax = fig.add_subplot(projection='polar')
ax.set_theta_offset(math.pi/2)
ax.set_theta_direction(-1)

for sat in sats:
    if sat.name.startswith("QZS"):
#    if sat.name.startswith("NAV"):
        print(sat.name)
        alt_azs = []
        difference = sat - home
        prev_appended = False
        for target_ts in tss:
            topocentric = difference.at(target_ts)
            alt, az, distance = topocentric.altaz()
            if 5 < alt.degrees:
                alt_azs.append({'alt':math.cos(alt.degrees/180 * math.pi), 'az':az.degrees/180 * math.pi})
                prev_appended = True
            else:
                if prev_appended and 0 < len(alt_azs):
                    df = pd.DataFrame(alt_azs)
                    ax.plot(df['az'], df['alt'], label=sat.name)
                    alt_azs = []
        if 0 < len(alt_azs):
            df = pd.DataFrame(alt_azs)
            if sat.name.startswith("QZS-3"):
                ax.plot(df['az'], df['alt'], label=sat.name, marker='o', markersize=10)
            else:
                ax.plot(df['az'], df['alt'], label=sat.name)
            alt_azs = []


plt.legend()
